In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv", delimiter=",")

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


# Business Understanding

The objective of this report is to analyze the factors that can contribute to the severity of an collision accident.

The severitycode column has 2 unique values "1" and "2" representing the severity of a collision accident where "1" = prop damage and "2" = injury.

We can use the attributes in the colomns to establish a relationship between the factors that contribute to a collision and its severity and use machine learning to predict the severity for given inputs

Target audience can be car navigation systems such as google maps, apple maps, waze, cars native navigantional systems and autonomous cars navigational and control systems

Through the given inputs, the model can predict whether your specific conditions could results in a severity type "1" or "2" accident and warn the user to adapt and implement necessary precautions

# Data understanding

First to understand the data, we find the dataframe's info which tells us all about the attributes, their types and how many non-null values each of them has
df.info() is shown below and has 37 different attributes and each attribute has up to 194673 values which is a good sample size. Take a look at the info table provided below

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
SEVERITYCODE      194673 non-null int64
X                 189339 non-null float64
Y                 189339 non-null float64
OBJECTID          194673 non-null int64
INCKEY            194673 non-null int64
COLDETKEY         194673 non-null int64
REPORTNO          194673 non-null object
STATUS            194673 non-null object
ADDRTYPE          192747 non-null object
INTKEY            65070 non-null float64
LOCATION          191996 non-null object
EXCEPTRSNCODE     84811 non-null object
EXCEPTRSNDESC     5638 non-null object
SEVERITYCODE.1    194673 non-null int64
SEVERITYDESC      194673 non-null object
COLLISIONTYPE     189769 non-null object
PERSONCOUNT       194673 non-null int64
PEDCOUNT          194673 non-null int64
PEDCYLCOUNT       194673 non-null int64
VEHCOUNT          194673 non-null int64
INCDATE           194673 non-null object
INCDTTM           194673 non-null obje

By examining the info table above, we observe some attributes that can be interesting and some that provide no use to our prediction of the severity of an accident. These attributes however do not provide the full picture because some of the attributes have NULL values, so data cleaning will be done before analysis and building the model to provide better model accurancy and an error free process. Interesting attributes to examine have been listed below

interesting attributes:
1. LOCATION : HAS NULL
2. WEATHER : HAS NULL
3. ROADCOND : HAS NULL
4. LIGHTCOND : HAS NULL
5. COLLISIONTYPE : HAS NULL
6. PEDCOUNT : FULL
7. PEDCYLCOUNT : FULL
8. VEHCOUNT : FULL
9. PERSONCOUNT : FULL
10. HITPARKEDCAR: FULL
11. SPEEDING: HAS NULL


We examine each of these attributes to look for modes or value patterns that they might have and see the general values each of these attributes have and how we can implement them in our model

In [8]:
df["LOCATION"].value_counts()

BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB AND AURORA AVE N                          276
BATTERY ST TUNNEL SB BETWEEN AURORA AVE N AND ALASKAN WY VI SB                          271
N NORTHGATE WAY BETWEEN MERIDIAN AVE N AND CORLISS AVE N                                265
AURORA AVE N BETWEEN N 117TH PL AND N 125TH ST                                          254
6TH AVE AND JAMES ST                                                                    252
AURORA AVE N BETWEEN N 130TH ST AND N 135TH ST                                          239
ALASKAN WY VI NB BETWEEN S ROYAL BROUGHAM WAY ON RP AND SENECA ST OFF RP                238
RAINIER AVE S BETWEEN S BAYVIEW ST AND S MCCLELLAN ST                                   231
WEST SEATTLE BR EB BETWEEN ALASKAN WY VI NB ON RP AND DELRIDGE-W SEATTLE BR EB ON RP    212
ALASKAN WY VI SB BETWEEN COLUMBIA ST ON RP AND ALASKAN WY VI SB EFR OFF RP              212
AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N                                      

In [9]:
df["WEATHER"].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [10]:
df["ROADCOND"].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [11]:
df["LIGHTCOND"].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [17]:
df["SEVERITYCODE"].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [14]:
df["COLLISIONTYPE"].value_counts()

Parked Car    47987
Angles        34674
Rear Ended    34090
Other         23703
Sideswipe     18609
Left Turn     13703
Pedestrian     6608
Cycles         5415
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

In [15]:
df["PEDCOUNT"].value_counts()

0    187734
1      6685
2       226
3        22
4         4
6         1
5         1
Name: PEDCOUNT, dtype: int64

In [18]:
df["JUNCTIONTYPE"].value_counts()

Mid-Block (not related to intersection)              89800
At Intersection (intersection related)               62810
Mid-Block (but intersection related)                 22790
Driveway Junction                                    10671
At Intersection (but not related to intersection)     2098
Ramp Junction                                          166
Unknown                                                  9
Name: JUNCTIONTYPE, dtype: int64

In [19]:
df["VEHCOUNT"].value_counts()

2     147650
1      25748
3      13010
0       5085
4       2426
5        529
6        146
7         46
8         15
9          9
11         6
10         2
12         1
Name: VEHCOUNT, dtype: int64

In [20]:
df["PEDCYLCOUNT"].value_counts()

0    189189
1      5441
2        43
Name: PEDCYLCOUNT, dtype: int64

In [21]:
df["PERSONCOUNT"].value_counts()

2     114231
3      35553
4      14660
1      13154
5       6584
0       5544
6       2702
7       1131
8        533
9        216
10       128
11        56
12        33
13        21
14        19
15        11
17        11
16         8
44         6
18         6
20         6
25         6
19         5
26         4
22         4
27         3
28         3
29         3
47         3
32         3
34         3
37         3
23         2
21         2
24         2
30         2
36         2
57         1
31         1
35         1
39         1
41         1
43         1
48         1
53         1
54         1
81         1
Name: PERSONCOUNT, dtype: int64

In [22]:
df["HITPARKEDCAR"].value_counts()

N    187457
Y      7216
Name: HITPARKEDCAR, dtype: int64

In [27]:
df["SPEEDING"].value_counts()

Y    9333
Name: SPEEDING, dtype: int64

Some NULL values in the attributes can be replaces with the mean or mode of all the values in it. Some values are not labaled as NULL such as in "JUNCTIONTYPE" attribute which has 9 "Unknown" values instead of NULL so those will be replaces by the mode. Some values are labaled as NULL instead of "NO" such as in "SPEEDING" attribute so all the NULL values will be replaced by "N"

Some attributes are redundant, We find in our interesting attributes that "PERSONCOUNT" is the total number of people involved in the collision which already includes PEDCOUNT and PEDCYLCOUNT. so we'll discard PEDCOUNT and PEDCYLCOUNT and only keep PERSONCOUNT in the analysis. this reduces our interesting attributes list to:

interesting attributes:
1. LOCATION : HAS NULL
2. WEATHER : HAS NULL
3. ROADCOND : HAS NULL
4. LIGHTCOND : HAS NULL
5. COLLISIONTYPE : HAS NULL
6. VEHCOUNT : FULL
7. PERSONCOUNT : FULL
8. HITPARKEDCAR: FULL
9. SPEEDING: HAS NULL


The ultimate goal thereafter is to train the model using a supervised Machine learning model to predict the severity of an accident with the given inputs